In [45]:
import pandas as pd 
import plotly.express as px 
import json
import plotly
from plotly.offline import plot

In [3]:
#import local csv with 2019 to 2021 population data (per state)
#dataset was joined and cleaned with SQL postgres and exported for visuals to be created here
df_pop = pd.read_csv("btu2019to2021export.csv")
df_pop

,st_abbreviation,2019,2020,2021,2022,2023,code,btu_2019,btu_2020,btu_2021
0,AK,733374,732964,734923,733276,733406,ABICB,-10.0,-7.0,-8.0
1,AK,733374,732964,734923,733276,733406,ARICB,11973.0,12391.0,13425.0
2,AK,733374,732964,734923,733276,733406,ARTCB,11973.0,12391.0,13425.0
3,AK,733374,732964,734923,733276,733406,ARTXB,11973.0,12391.0,13425.0
4,AK,733374,732964,734923,733276,733406,AVACB,1245.0,915.0,1037.0
...,...,...,...,...,...,...,...,...,...,...
12286,WY,576850,577664,579548,581629,584057,WYICB,0.0,0.0,0.0
12287,WY,576850,577664,579548,581629,584057,WYTCB,37045.0,48329.0,74713.0
12288,WY,576850,577664,579548,581629,584057,WYTXB,0.0,0.0,0.0
12289,WY,576850,577664,579548,581629,584057,ZWCDP,243.0,352.0,411.0


In [21]:
#RC is residential sector consumption
#filter energy use codes by RC, to get residential use (btu per state)
total_df_pop = df_pop[df_pop["code"].str.match("[A-Z]{2}RC[A-Z]")]
total_df_pop["code"].unique()

array(['CLRCB', 'DFRCB', 'ESRCB', 'GERCB', 'HLRCB', 'KSRCB', 'LORCB',
       'NGRCB', 'PARCB', 'PQRCB', 'RERCB', 'SFRCB', 'SORCB', 'TERCB',
       'TNRCB', 'WDRCB'], dtype=object)

In [22]:
#calculating use per person, per year
total_df_pop['2019pp']=total_df_pop['btu_2019']/total_df_pop['2019']
total_df_pop['2020pp']=total_df_pop['btu_2020']/total_df_pop['2020']
total_df_pop['2021pp']=total_df_pop['btu_2020']/total_df_pop['2021']

#create new dataframe with only 2019 to 2021 BTU/person data
df_covid = total_df_pop[['st_abbreviation','code','2019pp','2020pp','2021pp']]
#rename columns for simplification 
df_covid.columns=['state','code','2019','2020','2021']
df_covid

/var/folders/sh/lh8_sldj2tj11k741sl2jpnm0000gn/T/ipykernel_49812/2846017607.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/sh/lh8_sldj2tj11k741sl2jpnm0000gn/T/ipykernel_49812/2846017607.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/sh/lh8_sldj2tj11k741sl2jpnm0000gn/T/ipykernel_49812/2846017607.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,state,code,2019,2020,2021
27,AK,CLRCB,0.000000,0.000000,0.000000
35,AK,DFRCB,0.008964,0.009728,0.009702
55,AK,ESRCB,0.008971,0.009725,0.009699
71,AK,GERCB,0.000138,0.000138,0.000137
77,AK,HLRCB,0.000543,0.000595,0.000593
...,...,...,...,...,...
12234,WY,SFRCB,0.000000,0.000000,0.000000
12242,WY,SORCB,0.000106,0.000158,0.000157
12252,WY,TERCB,0.093494,0.090598,0.090303
12261,WY,TNRCB,0.056933,0.053580,0.053405


In [23]:
#use melt method to assign years per column of btu entry
#rearranging table will allow for plotting
df_covid_new= df_covid.melt(id_vars=["state", "code"], var_name="year", value_name="btu_per_person")
df_covid_new

,state,code,year,btu_per_person
0,AK,CLRCB,2019,0.000000
1,AK,DFRCB,2019,0.008964
2,AK,ESRCB,2019,0.008971
3,AK,GERCB,2019,0.000138
4,AK,HLRCB,2019,0.000543
...,...,...,...,...
2443,WY,SFRCB,2021,0.000000
2444,WY,SORCB,2021,0.000157
2445,WY,TERCB,2021,0.090303
2446,WY,TNRCB,2021,0.053405


In [24]:
#sum residential energy use by state and year
df_covid_grouped = df_covid_new.groupby(["state", "year"], as_index=False).sum()
df_covid_grouped


,state,year,code,btu_per_person
0,AK,2019,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.200405
1,AK,2020,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.224464
2,AK,2021,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.223866
3,AL,2019,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.171314
4,AL,2020,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.163031
...,...,...,...,...
145,WV,2020,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.220460
146,WV,2021,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.221240
147,WY,2019,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.263909
148,WY,2020,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.252394


In [26]:
df_covid_grouped.to_csv('covid_grouped.csv')

In [38]:
#Use animation frame to create interactive visual that demonstrates stage energy usage per person (in ascending order)
fig = px.bar(
        df_covid_grouped, 
        x="state", 
        y="btu_per_person", 
        animation_frame="year", 
        range_y=(
            0, 
            df_covid_grouped["btu_per_person"].max()
        ), 
        color="state",
        title="United States residential energy consumption (BTU)"
).update_xaxes(categoryorder="total ascending")

fig.show()

In [46]:
# Save the interactive HTML file
html_file_path = "interactive_plot.html"
plot(fig, filename=html_file_path, auto_open=False)

'interactive_plot.html'

In [39]:
#code below to convert figure to json did not work, output was not dynamic
#plotly_json_2 = fig.to_json()

In [35]:
#code below to convert figure to json did not work, output was not dynamic
# plotly_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)

In [47]:
# #code below to convert figure to json did not work, output was not dynamic
# print(plotly_json_2)

In [ ]:
#create dynamic bar plot with populations of state for each year
#create new dataframe with only 2019 to 2021 population data per state
df_statepop = total_df_pop[['st_abbreviation','code','2019pp','2020pp','2021pp']]